In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [2]:
#Stanting timming process
start = time.time()

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
#Rreading the name of the subfolder
subfolder_with_path = [f.path for f in os.scandir(main_directory) if f.is_dir()]

In [5]:
subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]

In [6]:
###############################################################################################
# Importing information from "samples overview", "intensisties template" and "brain allen map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])


# Reading "sample_overview.csv" file
samples_overview = pd.read_csv("sample_overview.csv", 
                               dtype={"MOUSE": "string","SAMPLE": int,
                                      "FILES_HD": int,"FILES_BOX": int,
                                      "HEMI": "string","STAIN": "string",
                                      "TX_GROUP": "string", 
                                      "GENDER":"string", 
                                      "CONTEXT":"string"})

# Reading "intensities_template.csv" file
intensities_template = pd.read_csv("intensities_template.csv",
                                   dtype={"LabelID": int, 
                                          "region": "string", 
                                          "labelabrv": "string"})
# Reading "brain_allen_map.csv" file
brain_allen_map = pd.read_csv("brain_allen_map.csv", 
                               dtype={"raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})


In [7]:
intensities_template

,LabelID,region,labelabrv
0,1,right_tuberomammillary_nucleus_ventral_part,rtmv
1,2,right_primary_somatosensory_area_mouth_layer_6b,rssp-m6b
2,4,right_inferior_colliculus,ric
3,6,right_internal_capsule,rint
4,7,right_principal_sensory_nucleus_of_the_trigeminal,rpsv
...,...,...,...
1106,21127,left_temporal_association_areas_layer_2/3,ltea2/3
1107,21128,left_primary_somatosensory_area_lower_limb_lay...,lssp-ll5
1108,21139,left_nucleus_of_the_lateral_olfactory_tract_la...,lnlot3
1109,30703,"left_dentate_gyrus,_molecular_layer",ldg-mo


In [ ]:
# Changing directory to the "02Volume_files" folder
os.chdir(subfolder_with_path[2])

#Counting number of files on "02Volume_files" folder
list = os.listdir(subfolder_with_path[2]) 
number_files = len(list)
number_volume_files = number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]


    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(6)

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
#     output['SAMPLE'] = sample
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)

In [ ]:
# Changing directory to the "03Volume_in_tissue_files" folder
os.chdir(subfolder_with_path[3])

#Counting number of files on "03Volume_in_tissue_files" folder
list = os.listdir(subfolder_with_path[3]) 
number_files = len(list)
number_volintissue_files = number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_intissue_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

sal_tdt = 0 # Counter of samples where Saline was used and tdTomato expressed
mdma_tdt = 0 # Counter of samples where MDMA was used and tdTomato expressed
meth_tdt = 0 # Counter of samples where Meth was used and tdTomato expressed

sal_cfos = 0 # Counter of samples where Saline was used and cfos expressed
mdma_cfos = 0 # Counter of samples where MDMA was used and cfos expressed
meth_cfos = 0 # Counter of samples where Meth was used and cfos expressed

#These array will contain sample's names as elements
samples_sal_tdt = [] 
samples_mdma_tdt = [] 
samples_meth_tdt = [] 
samples_sal_cfos = []
samples_mdma_cfos = [] 
samples_meth_cfos = []

    ##############################################
    # Processing each file
    ##############################################

while i < number_files:


    file = list[i]
    # Saving sample's number 
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME_IN_TISSUE'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME_IN_TISSUE'] = output['VOLUME_IN_TISSUE'].div(1000000).round(6)

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
#     output['SAMPLE'] = sample
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    #Adding a column with the sample's nickname

    if (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_sal_tdt:
            sal_tdt += 1
        samples_sal_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vSAL_' + str(sal_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_tdt:
            mdma_tdt += 1 
        samples_mdma_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vMDMA_' + str(mdma_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_meth_tdt:
            meth_tdt += 1 
        samples_meth_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vMETH_' + str(meth_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_sal_cfos:
            sal_cfos += 1 
        samples_sal_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vSAL_' + str(sal_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_cfos:
            mdma_cfos += 1 
        samples_mdma_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vMDMA_' + str(mdma_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_meth_cfos:
            meth_cfos += 1 
        samples_meth_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vMETH_' + str(meth_cfos)

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_intissue_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframeo
volume_intissue_data = pd.concat(volume_intissue_data)

In [ ]:
#calculating volume ratio
volumes_data = pd.merge(volume_intissue_data,volume_data[['raw','VOLUME','SAMPLE']],on=('raw','SAMPLE'),how='left')
volumes_data['volumes_ratio'] = volumes_data['VOLUME_IN_TISSUE'] / volumes_data['VOLUME'] 

#Pivoting dataframe to have nicknames as columns
pivoted_volumes_data = pd.pivot_table(volumes_data, values='VOLUME_IN_TISSUE', 
                                       index=['raw'],columns=['nickname'], aggfunc=np.sum)

In [8]:
# Changing directory to the "01Activity_files" folder
os.chdir(subfolder_with_path[1])

#Counting number of files on "01Activity_files" folder
list = os.listdir(subfolder_with_path[1]) 
number_files = len(list)
number_activity_files = number_files

In [9]:
#Using and timing  a loop to process all files and populate the main data-frame

activity_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

sal_tdt = 0 # Counter of samples where Saline was used and tdTomato expressed
mdma_tdt = 0 # Counter of samples where MDMA was used and tdTomato expressed
meth_tdt = 0 # Counter of samples where Meth was used and tdTomato expressed

sal_cfos = 0 # Counter of samples where Saline was used and cfos expressed
mdma_cfos = 0 # Counter of samples where MDMA was used and cfos expressed
meth_cfos = 0 # Counter of samples where Meth was used and cfos expressed

#These array will contain sample's names as elements
samples_sal_tdt = [] 
samples_mdma_tdt = [] 
samples_meth_tdt = [] 
samples_sal_cfos = []
samples_mdma_cfos = [] 
samples_meth_cfos = []


    ##############################################
    # Innitiates loop to process each file
    ##############################################


while i < number_files:


    file = list[i]
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]
    
#     # Saving rater's name
#     rater = file[8:10]


    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, 
                                                   ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
#     output['SAMPLE'] = sample
    
#     # Adding a column of the rater's name
#     rater_column = np.empty(len(output))
#     output['rater'] = rater_column
#     output['rater'] = rater
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    
    #Adding a column with the sample's nickname

    if (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_sal_tdt:
            sal_tdt += 1
        samples_sal_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'SAL_' + str(sal_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_tdt:
            mdma_tdt += 1 
        samples_mdma_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'MDMA_' + str(mdma_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_meth_tdt:
            meth_tdt += 1 
        samples_meth_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'METH_' + str(meth_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_sal_cfos:
            sal_cfos += 1 
        samples_sal_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'SAL_' + str(sal_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_cfos:
            mdma_cfos += 1 
        samples_mdma_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'MDMA_' + str(mdma_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_meth_cfos:
            meth_cfos += 1 
        samples_meth_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'METH_' + str(meth_cfos)

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname

    
    ##########################################################################################################################
    
    # store DataFrame in list
    activity_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
activity_data = pd.concat(activity_data)

In [13]:
file = list[0]
filename_segmented = file.split('_')
sample = filename_segmented[1]
regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
regions_count_activity_fracc

,PIXEL_COUNT,INTENSITY_1,INTENSITY_1_PERC,INTENSITY_2,INTENSITY_2_PERC,INTENSITY_3,INTENSITY_3_PERC
0,1,21017,1,0,0,0,0
1,3,21017,1,0,0,0,0
2,1,21017,1,0,0,0,0
3,4,21017,1,0,0,0,0
4,1,21017,1,0,0,0,0
...,...,...,...,...,...,...,...
636591,3,776,1,0,0,0,0
636592,1,776,1,0,0,0,0
636593,2,588,1,0,0,0,0
636594,1,296,1,0,0,0,0


In [10]:
activity_data.to_csv('activity_data.csv', index=False)
activity_data

,LabelID,region,labelabrv,ACTIVITY,SAMPLE,GENDER,hemisphere,marker,TX_GROUP,CONTEXT,nickname
0,1,right_tuberomammillary_nucleus_ventral_part,rtmv,NaN,2.0,F,left,cfos,Saline,group housed,SAL_1
1,2,right_primary_somatosensory_area_mouth_layer_6b,rssp-m6b,NaN,2.0,F,left,cfos,Saline,group housed,SAL_1
2,4,right_inferior_colliculus,ric,101.0,2.0,F,left,cfos,Saline,group housed,SAL_1
3,6,right_internal_capsule,rint,NaN,2.0,F,left,cfos,Saline,group housed,SAL_1
4,7,right_principal_sensory_nucleus_of_the_trigeminal,rpsv,NaN,2.0,F,left,cfos,Saline,group housed,SAL_1
...,...,...,...,...,...,...,...,...,...,...,...
1106,21127,left_temporal_association_areas_layer_2/3,ltea2/3,1024.0,2.0,F,left,cfos,Saline,group housed,SAL_1
1107,21128,left_primary_somatosensory_area_lower_limb_lay...,lssp-ll5,4131.0,2.0,F,left,cfos,Saline,group housed,SAL_1
1108,21139,left_nucleus_of_the_lateral_olfactory_tract_la...,lnlot3,1539.0,2.0,F,left,cfos,Saline,group housed,SAL_1
1109,30703,"left_dentate_gyrus,_molecular_layer",ldg-mo,5682.0,2.0,F,left,cfos,Saline,group housed,SAL_1


In [ ]:
#Replacing empty cells with zeros
activity_data['ACTIVITY'].fillna(0, inplace=True)

In [ ]:
#Merging activity and volume ratio to filter activity where the volume ratio is below 50%
filtered_activity_data = pd.merge(activity_data,
                                  volumes_data[['raw','SAMPLE','volumes_ratio']],
                                  on=['raw','SAMPLE'],
                                  how='left')
filtered_activity_data.loc[(filtered_activity_data['volumes_ratio'] <= 0.5) | (filtered_activity_data['volumes_ratio'] > 1.0)
                           ,'ACTIVITY'] = ''

In [ ]:
#Pivoting dataframe to have nicknames as columns
pivoted_activity_data = pd.pivot_table(filtered_activity_data, values='ACTIVITY', 
                               index=['raw','hemisphere','marker'],columns=['nickname'], aggfunc=np.sum)

#Restarting index to activate'hemisphere','marker'and 'rater' as columns
pivoted_activity_data = pivoted_activity_data.reset_index()

In [ ]:
# Changing directory to the main path
os.chdir(main_directory)

In [ ]:
#combining dataframes to create the "cell_count_output.csv" file
allen_and_activity = pd.merge(brain_allen_map,pivoted_activity_data,on='raw',how='right')
cell_count_output = pd.merge(allen_and_activity,pivoted_volumes_data,on='raw',how='left')
cell_count_output.to_csv('cell_count_output.csv', index=False)

In [ ]:
#Ending timming process
end = time.time()

# Printing how many files were processed and how much time the process took
files_processed = number_activity_files = number_files+ number_volume_files + number_volintissue_files
print(files_processed,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 